In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import time
import scipy
path = '/home/rinnely/ENES/2019-2/Opti_App/informacionRutas/informacionRutas/sprint7ToroideMixto.csv'
global data, lat, lon, fig, f
import BentleyWithHV as BHV
import pygmo as pg
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns


In [3]:
def getData(path):
    data = pd.read_csv(path)
    data['tiempo'] = data['demanda']*data['frecuencia']
    return data[['lat', 'lon', 'tiempo']]

In [4]:
def updateClusters(particula):
    global data, lat,lon
    
    #si se sale de los limites eligo un punto al azar
    
    for i, position in enumerate(particula.position):
        particula.tiempo.append(0)
        #if f != 0:
         #3   if particula.points[i]:
           #     puntos = pd.DataFrame(particula.points[i], columns=['lat', 'lon'])
            #    lat = np.array([min(puntos['lat']), max(puntos['lat'])])
             #   lon = np.array([min(puntos['lon']), max(puntos['lon'])])
            #else:
             #   lat = np.array([min(data['lat']), max(data['lat'])])
              #  lon = np.array([min(data['lon']), max(data['lon'])])
        #else:
        #lat = np.array([min(data['lat']), max(data['lat'])])
        #lon = np.array([min(data['lon']), max(data['lon'])])
            
        if position[0] < lat[0] or position[0] > lat[1]:
            particula.position[i] = data[['lat','lon']].sample(1)

        if position[1] < lon[0] or position[1] > lon[1]:
            particula.position[i] = data[['lat','lon']].sample(1)

        particula.points[i] = []
        
    for point in data.iterrows():
        
        punto = np.array(point[1][:2])
        
        #punto = point[1][['lat','lon']]
        
        dist = scipy.spatial.distance.cdist(particula.position, punto*np.ones_like(particula.position), 'euclidean')
        #dist_euc = dist[dist.argmin(axis=0)[0]][0]

        centroid = dist.argmin(axis=0)[0]
                
        particula.points[centroid].append(punto)                
        particula.tiempo[centroid] += point[1].tiempo
        

In [5]:
def getFitnessTiempo(particula):
    particula.fitness[0] = np.std(particula.tiempo)

In [6]:
def getAllFitness(swarm):
    all_fitnes = []
    all_position = []
    index = 0
    min_fit = 10e10
    i = 0
    for particula in swarm:
        if particula.best_fitness[0] <= min_fit:
            min_fit = particula.best_fitness[0]
            index = i
        i += 1
    return min_fit, index

In [7]:
def getAllAreas(swarm):
    all_areas = []
    index = 0
    min_fit = 10e10
    i = 0
    
    for particula in swarm:
        if particula.best_area[1] <= min_fit:
            min_fit = particula.best_area[1]
            index = i
        i += 1
    return min_fit, index

In [8]:
def getFitnessArea(particula):
    
    particula.areas = []
    
    for i, position in enumerate(particula.position):
        if particula.points[i]:
            centroid = position*np.ones_like(particula.points[i])      

            dist = scipy.spatial.distance.cdist(particula.points[i], centroid, 'euclidean')
            r = dist[dist.argmax(axis=0)[0]][0]
            area = np.pi * (r*r)
        else:
            area = 0

        particula.areas.append(area)

    particula.fitness[1] = np.std(particula.areas)

In [9]:
def plotContribution(nondom, z_ref):
    global ax2
    ax2.clear()    
    

    ax2.scatter(z_ref[0], z_ref[1], label=(z_ref))
    
    for px,py in nondom:
        #print('punto:', punto)
        ax2.scatter(px, py, label=(px,py))
        #plt.annotate(punto, (punto[0], punto[1]))
        x = np.linspace(px,z_ref[0])
        y = np.linspace(py,z_ref[1])
        a = np.ones_like(x)*py
        b = np.ones_like(y)*px
        ax2.plot(x, a, '--')
        ax2.plot(b, y, '--')

    x = np.linspace(nondom[0][0], z_ref[0])
    y = np.linspace(z_ref[1], sorted(nondom, key=lambda s : s[1])[0][1])
    a = np.ones_like(x)*z_ref[1]
    b = np.ones_like(y)*z_ref[0]
    ax2.plot(x,a, '--')
    ax2.plot(b, y, '--')
    ax2.set_title('Dominancia')
    #plt.title(best_global_fitness)
    #ax2.legend()
    plt.pause(0.01)
    #plt.show()

In [10]:
def getContributions(swarm, fitness):
    contributions = []
    
    #nondom = BHV.bentley(fitness)
    pareto = pg.non_dominated_front_2d(fitness)
    nondom = [tuple(fitness[i]) for i in pareto]
    nondom = sorted(nondom, key=lambda s : s[0])
    #ordeno nondom
    #nondom = np.array(nondom)
    #print('nondom', nondom)
    #nondom.view('i8,i8').sort(order=['f0'], axis=0)
    #print('nondom-sort', nondom)
    
    z_ref = np.array([nondom[-1][0]+.015, sorted(nondom, key=lambda s : s[1])[-1][1]+.015])
    plotContribution(nondom, z_ref)
    #calculo contribucion
    #plotContribution(nondom, z_ref)
    #nondom = nondom.tolist()
    
    for i, particula in enumerate(swarm):
        if tuple(particula.fitness) not in nondom:
            particula.contribution = 0
                
        else:
            idx = nondom.index(tuple(particula.fitness))
            particula.contribution = BHV.hipervolumen(nondom, idx, z_ref)
            contributions.append((particula.contribution, i))

    contributions.sort()
    return contributions

In [11]:
def plotClusters(particula):
    global ax1, best_global_fitness
    ax1.clear()
    for i in particula.points:
        
        if len(particula.points[i]) > 1:
            #a = sns.color_palette("gnuplot2", 13)
            a = sns.color_palette("Set2", 13)

            p = pd.DataFrame(particula.points[i], columns=['x','y'])
            ax1.plot(p.x, p.y, '.', color=a[i], label=(i, len(p)) )
            ax1.plot(particula.position[i][0], particula.position[i][1], '*', color=a[i], markersize=13, markeredgecolor='w')
            #ax.annotate(i, (particula.position[i][0], particula.position[i][1]))
            #
    ax1.set_title('Clusters')
    #ax1.set_title(best_global_fitness)
    ax1.legend()
    plt.pause(0.001)
    #plt.show()
    

In [12]:
class Particle:
    global data
    def __init__(self, k):
        self.position = np.array(data[['lat','lon']].sample(n=k, random_state=random.randint(0,100)))
        self.fitness = [10e10,10e10]
        self.best_fitness = [10e10,10e10]
        #self.data = data
        self.k = k
        self.best_position = []
        self.tiempo = []
        self.areas = []
        self.points = {}
        self.contribution = 0
        

In [13]:
def PSO(k, poblacion, ciclos):
    global fig, ax1, ax2, best_global_fitness
    #global data
    #x := current position
    #v := current velocity
    #y := best_position
    #w := weight innertia
    %matplotlib qt
    sns.set(style="ticks")
    plt.style.use("dark_background")
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
    plt.ion()
    c1 = c2 = 1.49
    r1 = random.uniform(0,1)
    r2 = random.uniform(0,1)
    w = .72
    velocity = np.array([1e-4,1e-4])
    best_global_fitness = []
    best_global_position = []
    #contributions = []
    
    fitness = []
    #f = 0
    #genero poblacion
    #print('generando poblacion...')
    swarm = []
    
    for i in range(poblacion):
        swarm.append(Particle(k))
        updateClusters(swarm[i])
        getFitnessTiempo(swarm[i])
        getFitnessArea(swarm[i])
        swarm[i].best_position = swarm[i].position
        fitness.append(swarm[i].fitness)
        

    #f = 1
    #print('\npoblacion generada')
    
    #algoritmo de bentley  
    
    contributions = getContributions(swarm, fitness)
    #print('cont:', contributions)
    
    #mejor posicion global:= el mejor contribucion de los nondom
    if len(contributions)>1:
        p = random.choice(contributions[int(len(contributions)*.80):])[1]
    else:
        p = contributions[0][1]
        
    best_global_position = swarm[p].position
    best_global_fitness = swarm[p].fitness
    #print('best_global_position:',best_global_position)
    #print('best_global_fitness:', best_global_fitness)
    #print()
    #plotClusters(swarm[p])
    #print('iniciando ciclos')
    for ciclo in range(ciclos):
        #start = time.time()

        fitness = []
        for particula in swarm:
            
            if len(contributions)>1:
                p = random.choice(contributions[:int(len(contributions)*.80)])[1]
            else:
                p = contributions[0][1]
            
            position = swarm[p].position
            
        #actualizo velocidad y posicion
        
            veloicty = (w*velocity + (c1*r2*(position - particula.position)) + (c2*r2 *(best_global_position - particula.position)))
            
            particula.position += velocity
            
            particula.tiempo = []
            particula.areas = []
            
                        
            #si se sale de los limites eligo un punto al azar
                    
            #actualizo clusters
            updateClusters(particula)
            
            plotClusters(particula)
            #plt.pause(0.1)
            #plt.show()
            #actualizo fitness
            getFitnessTiempo(particula)
            getFitnessArea(particula)
            
            fitness.append(particula.fitness)
        #algoritmo de bentley   
        
        plotClusters(swarm[p])

        contributions = getContributions(swarm, fitness)
        #print('contributions:', contributions)

        #mejor posicion global:= el mejor contribucion de los nondom
        
        if len(contributions)>1:
            p = random.choice(contributions[int(len(contributions)*.80):])[1]
        else:
            p = contributions[0][1]

        best_global_position = swarm[p].position
        best_global_fitness = swarm[p].fitness
#         print('best_global_position:',best_global_position)
        #print('best_global_fitness:', best_global_fitness)
        w -= 0.001
        
        #print('tiempo ciclo %d: %d segs. \n' %(ciclo, (time.time()-start)))
        #print()
    #return best_global_fitness, best_global_position
    return best_global_fitness, best_global_position
        
    

In [14]:
def runPSO(corridas,k,poblacion, ciclos):
    stds = []
    best_position = []
    full_time = time.time()
    tiempo = []

    for i in range(corridas):
        #print('corrida %d' %i)
        start = time.time()
        bg, bp, = PSO(k,poblacion, ciclos)
        stds.append(bg)
        best_position.append(bp)
        tiempo.append(time.time() - start)
        #print('time: %d segs \n' % (time.time() - start))

    print('total time: %d' % (time.time() - full_time))  
    #mean = np.mean(stds)
    #std = np.std(std)
    return stds, best_position, tiempo

In [15]:
data = getData(path)
lat = np.array([min(data['lat']), max(data['lat'])])
lon = np.array([min(data['lon']), max(data['lon'])])

In [16]:
#stds, best_position, tiempo = runPSO(30,13, 20, 100)

In [17]:
#tomar el centroide, encuentro el punto mas alejado de ese centroide, calculo el area, y calculo la desviación estandar

In [18]:
#particula = Particle(13)

In [19]:
#updateClusters(particula)
#getFitnessTiempo(particula)
#getFitnessArea(particula)

In [20]:
# particula.fitness

In [16]:
best_global_fitness, best_global_position = PSO(13, 30,100 )

KeyboardInterrupt: 